### Generating Rabindranath Tagore's Short Story, Shanskar

##### MD Muhaimin Rahman


contact: sezan92[at]gmail[dot]com

In this project I am trying to generate some lines from the Short Story of Rabindranath Tagore, named [Shanskar](http://chhotogalpo.com/%e0%a6%b8%e0%a6%82%e0%a6%b8%e0%a7%8d%e0%a6%95%e0%a6%be%e0%a6%b0/) , which is in Bangla. It is a very simple project of Natural Language Processing, which requires Recurrent Neural Network. It will learn the story from the text file. Then It will generate 100 lines from a random sentence from the Story. Many complex things can be done, I am willing to work on it more in the future God willing . This project is inspired by [this blog](https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/)

Importing Libraries

In [1]:
from __future__ import print_function
import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional
from keras.layers import TimeDistributed, Activation, SimpleRNN, GRU
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.regularizers import l2, l1
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam

from keras.metrics import categorical_crossentropy, categorical_accuracy
from keras.layers.convolutional import *
from keras.preprocessing import image, sequence
from keras.preprocessing.text import Tokenizer
import numpy as np



Using TensorFlow backend.


Importing text file ```Rabi_Shanskar.txt``` . Then decoding it to unicode as I am using Python 2

In [2]:
text = open('Rabi_Shanskar.txt').read()
text = text.decode('utf8')

In [3]:
print(text[0:1000])

চিত্রগুপ্ত এমন অনেক পাপের হিসাব বড়ো অক্ষরে তাঁর খাতায় জমা করেন যা থাকে পাপীর নিজের অগোচরে। তেমনি এমন পাপও ঘটে যাকে আমিই চিনি পাপ ব’লে, আর কেউ না। যেটার কথা লিখতে বসেছি সেটা সেই জাতের। চিত্রগুপ্তের কাছে জবাবদিহি করবার পূর্বে আগে-ভাগে কবুল করলে অপরাধের মাত্রাটা হাল্কা হবে।

ব্যাপারটা ঘটেছিল কাল শনিবার দিনে। সেদিন আমাদের পাড়ায় জৈনদের মহলে কী-একটা পরব ছিল। আমার স্ত্রী কলিকাকে নিয়ে মোটরে করে বেরিয়েছিলুম— চায়ের নিমন্ত্রণ ছিল বন্ধু নয়নমোহনের বাড়িতে।

স্ত্রীর কলিকা নামটি শ্বশুর-দত্ত, আমি ওর জন্য দায়ী নই। নামের উপযুক্ত তাঁর স্বভাব নয়, মতামত খুবই পরিস্ফুট। বড়োবাজারে বিলিতি কাপড়ের বিপক্ষে যখন পিকেট করতে বেরিয়েছিলেন, তখন দলের লোক ভক্তি ক’রে তাঁর নাম দিয়েছিল ধ্রুবব্রতা। আমার নাম গিরীন্দ্র দলের লোক আমাকে আমার পত্নীর পতি ব’লেই জানে, স্বনামের সার্থকতার প্রতি লক্ষ্য করে না। বিধাতার কৃপায় পৈতৃক উপার্জনের গুণে আমারও কিঞ্চিৎ সার্থকতা আছে। তার প্রতি দলের লোকের দৃষ্টি পড়ে চাঁদা আদায়ের সময়।

স্ত্রীর সঙ্গে স্বামীর স্বভাবের অমিল থাকলেই মিল ভালো হয়, শুকনো মাটির সঙ্গে জলধারার মতো। আমার প্রকৃতি অত্যন্ত ঢিলে, কিছু

In [4]:
words = sorted(text.split())
vocab_size = len(words)+1

print('total words:', vocab_size)


total words: 1395


In [5]:
word_indices = dict((w,i) for i,w in enumerate(words))
indices_words = dict((i,w) for i,w in enumerate(words))


In [6]:
w_idx = [word_indices[w] for w in text.split()]
print("idx")
len(w_idx)


idx


1394

Here I am taking 8 words per sentance. You may change it

In [7]:
cs =8


In [8]:
c_in_dat=[]
for n in range(0,len(w_idx)-1-cs,cs):
    c_in_dat.append([w_idx[i+n] for i in range(cs)])
c_out_dat = []
for i in range(cs,len(w_idx)-1,cs):
    c_out_dat.append(w_idx[i])


In [9]:
c_in_dat


[[443, 209, 14, 834, 1349, 1016, 0, 546],
 [385, 484, 290, 1123, 596, 833, 751, 3],
 [586, 209, 832, 418, 1126, 135, 445, 831],
 [1021, 141, 362, 726, 1156, 240, 1188, 937],
 [1292, 1285, 490, 444, 319, 482, 259, 862],
 [46, 244, 262, 25, 1068, 1345, 1331, 1014],
 [419, 341, 1196, 628, 1295, 88, 851, 504],
 [1062, 360, 812, 475, 114, 1298, 310, 775],
 [1107, 282, 999, 442, 756, 474, 910, 799],
 [962, 1302, 308, 738, 1210, 134, 236, 479],
 [621, 703, 740, 163, 546, 1312, 793, 1051],
 [387, 819, 1017, 975, 332, 972, 1119, 852],
 [257, 1000, 525, 617, 1190, 1027, 377, 546],
 [736, 642, 700, 114, 736, 402, 617, 1190],
 [80, 114, 806, 805, 1025, 494, 1311, 1263],
 [878, 1182, 282, 726, 969, 361, 870, 167],
 [407, 117, 351, 1262, 56, 578, 878, 617],
 [1192, 651, 893, 436, 62, 1249, 1302, 1238],
 [1320, 1316, 33, 593, 1083, 1035, 1360, 1202],
 [1064, 1238, 488, 1053, 114, 875, 8, 521],
 [346, 1006, 377, 452, 685, 782, 114, 1302],
 [875, 8, 41, 1123, 688, 539, 349, 468],
 [726, 88, 181, 1007, 

In [10]:
x = np.stack(c_in_dat)

In [11]:
x.shape

(174, 8)

In [12]:
n_fac =42
n_hidden=512


c_out_seq=[]
for i in range(1,len(c_in_dat)):
    c_out_seq.append(c_in_dat[i])
c_out_seq.append(c_in_dat[0])

In [13]:
y_seq = np.stack(c_out_seq)

Making the Sequential output Three Dimensional

In [14]:
y_seq=np.atleast_3d(y_seq)

In [15]:
y_seq.shape


(174, 8, 1)

In [16]:
model_sequence = Sequential()
model_sequence.add(Embedding(vocab_size,n_fac,input_length=cs))
model_sequence.add(LSTM(n_hidden,return_sequences=True,activation='tanh'))
model_sequence.add(TimeDistributed(Dense(vocab_size,activation='softmax')))
model_sequence.compile(loss='sparse_categorical_crossentropy',optimizer=Adam())
model_sequence.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 8, 42)             58590     
_________________________________________________________________
lstm_1 (LSTM)                (None, 8, 512)            1136640   
_________________________________________________________________
time_distributed_1 (TimeDist (None, 8, 1395)           715635    
Total params: 1,910,865
Trainable params: 1,910,865
Non-trainable params: 0
_________________________________________________________________


In [17]:
model_sequence.fit(x,y_seq,batch_size=64,epochs=1000)


Epoch 1/1000
174/174 [==============================] - 6s 36ms/step - loss: 7.2405
Epoch 2/1000
174/174 [==============================] - 0s 2ms/step - loss: 7.2365
Epoch 3/1000
174/174 [==============================] - 0s 2ms/step - loss: 7.2307
Epoch 4/1000
174/174 [==============================] - 0s 2ms/step - loss: 7.2178
Epoch 5/1000
174/174 [==============================] - 0s 2ms/step - loss: 7.1768
Epoch 6/1000
174/174 [==============================] - 0s 2ms/step - loss: 7.0367
Epoch 7/1000
174/174 [==============================] - 0s 2ms/step - loss: 6.8567
Epoch 8/1000
174/174 [==============================] - 0s 2ms/step - loss: 6.7591
Epoch 9/1000
174/174 [==============================] - 0s 2ms/step - loss: 6.6735
Epoch 10/1000
174/174 [==============================] - 0s 2ms/step - loss: 6.6553
Epoch 11/1000
174/174 [==============================] - 0s 2ms/step - loss: 6.6446
Epoch 12/1000
174/174 [==============================] - 0s 2ms/step - loss: 6.6245


174/174 [==============================] - 0s 2ms/step - loss: 1.5094
Epoch 99/1000
174/174 [==============================] - 0s 2ms/step - loss: 1.4732
Epoch 100/1000
174/174 [==============================] - 0s 2ms/step - loss: 1.4349
Epoch 101/1000
174/174 [==============================] - 0s 2ms/step - loss: 1.3980
Epoch 102/1000
174/174 [==============================] - 0s 2ms/step - loss: 1.3623
Epoch 103/1000
174/174 [==============================] - 0s 2ms/step - loss: 1.3250
Epoch 104/1000
174/174 [==============================] - 0s 2ms/step - loss: 1.2887
Epoch 105/1000
174/174 [==============================] - 0s 2ms/step - loss: 1.2596
Epoch 106/1000
174/174 [==============================] - 0s 2ms/step - loss: 1.2292
Epoch 107/1000
174/174 [==============================] - 0s 2ms/step - loss: 1.2066
Epoch 108/1000
174/174 [==============================] - 0s 2ms/step - loss: 1.1749
Epoch 109/1000
174/174 [==============================] - 0s 2ms/step - loss: 1.1

174/174 [==============================] - 0s 2ms/step - loss: 0.1923
Epoch 195/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.1889
Epoch 196/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.1854
Epoch 197/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.1821
Epoch 198/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.1790
Epoch 199/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.1758
Epoch 200/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.1727
Epoch 201/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.1697
Epoch 202/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.1668
Epoch 203/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.1638
Epoch 204/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.1609
Epoch 205/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.

174/174 [==============================] - 0s 2ms/step - loss: 0.0455
Epoch 291/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0452
Epoch 292/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0447
Epoch 293/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0443
Epoch 294/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0438
Epoch 295/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0435
Epoch 296/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0431
Epoch 297/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0427
Epoch 298/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0424
Epoch 299/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0420
Epoch 300/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0416
Epoch 301/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.

174/174 [==============================] - 0s 2ms/step - loss: 0.0269
Epoch 387/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0269
Epoch 388/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0268
Epoch 389/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0267
Epoch 390/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0267
Epoch 391/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0265
Epoch 392/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0266
Epoch 393/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0265
Epoch 394/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0264
Epoch 395/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0262
Epoch 396/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0263
Epoch 397/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.

...

174/174 [==============================] - 0s 2ms/step - loss: 0.0197
Epoch 579/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0198
Epoch 580/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0198
Epoch 581/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0198
Epoch 582/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0197
Epoch 583/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0197
Epoch 584/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0197
Epoch 585/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0197
Epoch 586/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0196
Epoch 587/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0196
Epoch 588/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0196
Epoch 589/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.

174/174 [==============================] - 0s 2ms/step - loss: 0.0184
Epoch 675/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0185
Epoch 676/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0185
Epoch 677/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0185
Epoch 678/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0186
Epoch 679/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0185
Epoch 680/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0185
Epoch 681/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0184
Epoch 682/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0185
Epoch 683/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0185
Epoch 684/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0184
Epoch 685/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.

174/174 [==============================] - 0s 2ms/step - loss: 0.0177
Epoch 771/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0179
Epoch 772/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0176
Epoch 773/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0176
Epoch 774/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0176
Epoch 775/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0177
Epoch 776/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0177
Epoch 777/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0177
Epoch 778/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0177
Epoch 779/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0178
Epoch 780/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0176
Epoch 781/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.

174/174 [==============================] - 0s 2ms/step - loss: 0.0173
Epoch 867/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0172
Epoch 868/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0172
Epoch 869/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0172
Epoch 870/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0172
Epoch 871/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0173
Epoch 872/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0172
Epoch 873/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0172
Epoch 874/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0171
Epoch 875/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0172
Epoch 876/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0172
Epoch 877/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.

174/174 [==============================] - 0s 2ms/step - loss: 0.0168
Epoch 963/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0168
Epoch 964/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0168
Epoch 965/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0168
Epoch 966/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0168
Epoch 967/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0168
Epoch 968/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0168
Epoch 969/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0168
Epoch 970/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0168
Epoch 971/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0168
Epoch 972/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.0167
Epoch 973/1000
174/174 [==============================] - 0s 2ms/step - loss: 0.

Random Input Sequence. At 13th February , 2018, GMT+6

In [20]:
start = np.random.randint(0,len(c_in_dat))
inp_text = c_in_dat[start]
for wi in inp_text:
    print(words[wi],end=' ')

সময় এই জায়গাটাতে এসে ঠেকে গেল। শুনতে পেলেম 

Predicted 10 Sentences

In [21]:
for i in range(10):
    arr = np.array(inp_text)[np.newaxis,:]
    p = model_sequence.predict(arr)[0]
    output = [words[np.argmax(o)] for o in p]
    for word in output:
        print(word, end=' ')
    inp_text = [word_indices[word] for word in output]
    

মার-মার ধ্বনি। মনে ভাবলুম, কোনো গাঁটকাটাকে শাসন চলছে। মোটরের শিঙা ফুঁকতে ফুঁকতে উত্তেজিত জনতার কেন্দ্রের কাছে গিয়ে দেখি, আমাদের পাড়ার বুড়ো সরকারি মেথরটাকে বেদম মারছে। একটু আগেই রাস্তার কলতলায় স্নান সেরে সাফ কাপড় পরে ডান হাতে এক বালতি জল ও বগলে ঝাঁটা নিয়ে রাস্তা দিয়ে সে যাচ্ছিল। গায়ে চেক-কাটা মেরজাই, আঁচড়ানো চুল ভিজে; বাঁ হাত ধরে সঙ্গে চলেছিল আট-নয় বছরের এক নাতি। দুজনকেই দেখতে সুশ্রী, সুঠাম দেহ। সেই ভিড়ে কারও সঙ্গে বা কিছুর সঙ্গে তাদের ঠেকাঠেকি হয়ে থাকবে। তার থেকে 

The Real Text

![Real](Shanskar.png)

In [22]:
model_seq_json = model_sequence.to_json()
with open("Model_Rabi_Sequence.json", "w") as json_file:
    json_file.write(model_seq_json)
# serialize weights to HDF5
model_sequence.save_weights("model_Rabi_Sequence.h5")
print("Saved model to disk")

Saved model to disk


In [24]:
!jupyter notebook list

Currently running servers:
http://localhost:8888/?token=80d813546431b85d8e8f9d22d11ae018b9cfad604c58a0b6 :: /home/sezan92
